In [1]:
#Import packages 
import time
import re
import string 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located

from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter

In [2]:
#Load workbook data
wb = load_workbook(filename = 'input.xlsx')
sheet = wb.active

In [3]:
destinations = []
for cell in ['B1', 'C1', 'D1', 'E1']:
    destinations.append(sheet[cell].value)

In [4]:
def click_train_btn(driver: webdriver.Chrome, wait: WebDriverWait):        
    ## Find train btn
    wait.until(
        presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[8]/div[3]/div[1]/div[2]/div/div[2]/div/div/div/div[3]/button')))
    btn = driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div[8]/div[3]/div[1]/div[2]/div/div[2]/div/div/div/div[3]/button')[0]
    ## Click train btn
    btn.click()
    
def scrape_duration(wait: WebDriverWait):
    ## Get time duration in 1st suggested trip, by using the XPATH of the 1st time element
    d1 = wait.until(
        presence_of_element_located((By.XPATH, '//*[@id="section-directions-trip-0"]/div[1]/div/div[1]/div')))

    return d1.text

In [5]:
#Start web scraping
print("Starting Chrome....")
with webdriver.Chrome() as driver:
    wait = WebDriverWait(driver, 10)
    
    row = 2  #starting from row 2 in excel
    
    while row < 501:
        origin = sheet['A'+ str(row)].value
        if origin != None:
            for i in range(len(destinations)): 
                # Open directions url
                directions_url = f"https://www.google.com/maps/dir/{origin}/{destinations[i]}"
                driver.get(directions_url)
                
                click_train_btn(driver, wait)
                time.sleep(0.1) #wait for new trip results after clicking train btn
                
                duration = scrape_duration(wait)
                print(f"{directions_url}: {duration}")
                
                # Write duration into excel
                sheet[string.ascii_uppercase[i+1]+ str(row)] = duration
        else:
            break
        row+=1    

    wb.save("./output.xlsx")

Starting Chrome....
https://www.google.com/maps/dir/東京都新宿区北新宿2-19-2-302/中目黒駅: 31 min
https://www.google.com/maps/dir/東京都新宿区北新宿2-19-2-302/渋谷駅: 24 min
https://www.google.com/maps/dir/東京都新宿区北新宿2-19-2-302/代官山駅: 30 min
https://www.google.com/maps/dir/東京都新宿区北新宿2-19-2-302/恵比寿駅: 27 min
https://www.google.com/maps/dir/十条駅/中目黒駅: 26 min
https://www.google.com/maps/dir/十条駅/渋谷駅: 16 min
https://www.google.com/maps/dir/十条駅/代官山駅: 27 min
https://www.google.com/maps/dir/十条駅/恵比寿駅: 18 min
